# Interactive map plots for MovingPandas

An evaluation of **folium** and **hvplot**

In [ ]:
%matplotlib inline

In [ ]:
import urllib
import os
import pandas as pd
import geopandas as gpd
import contextily as ctx
from geopandas import GeoDataFrame, read_file
from shapely.geometry import Point, LineString, Polygon
from datetime import datetime, timedelta
from matplotlib import pyplot as plt

import sys
sys.path.append("..")
import movingpandas as mpd

import warnings
warnings.simplefilter("ignore")

In [ ]:
df = read_file('data/demodata_geolife.gpkg')
df['t'] = pd.to_datetime(df['t'])
df = df.set_index('t').tz_localize(None)
print("Finished reading {} rows".format(len(df)))

In [ ]:
traj_collection = mpd.TrajectoryCollection(df, 'trajectory_id')
print(traj_collection)

In [ ]:
gdf = traj_collection.trajectories[1].df

In [ ]:
import hvplot.pandas
from holoviews import opts
import folium
import branca

## Minimum viable

For tile options available in hvplot, check https://hvplot.pyviz.org/user_guide/Geographic_Data.html

In [ ]:
gdf.hvplot(geo=True, tiles=True, width=500, height=300) 

In [ ]:
folium_map = folium.Map()

for pt in gdf.geometry:
    folium.CircleMarker(
        [pt.y, pt.x]
    ).add_to(folium_map)

folium_map

## Standard interaction and zoom to area of interest

In [ ]:
opts.defaults(
    opts.Overlay(
        width=500, height=300,
        active_tools=['wheel_zoom']
))

gdf.hvplot(geo=True, tiles=True) 

In [ ]:
folium_map = folium.Map([40.02, 116.45], zoom_start=11)

for pt in gdf.geometry:
    folium.CircleMarker(
        [pt.y, pt.x],
        radius=2, fill=True
    ).add_to(folium_map)

folium_map

## Color by attribute

In [ ]:
gdf.hvplot(geo=True, tiles=True, c='sequence') 

In [ ]:
folium_map = folium.Map([40.02, 116.45], zoom_start=11)
colorscale = branca.colormap.linear.YlGnBu_09.scale(1000,2000)

for pt, seq in gdf[['geometry','sequence']].values:
    folium.CircleMarker(
        [pt.y, pt.x],
        radius=3, fill=True,
        color='b', fill_color=colorscale(seq),
        fill_opacity=1.0
    ).add_to(folium_map)

folium_map

## Drawing lines instead of points

To replicate the built-in Trajectory.plot() function, we have to plot the Trajectory's line representation rather than its points.

In [ ]:
temp = traj_collection.trajectories[1].copy()
temp.add_speed(overwrite=True)
line_gdf = temp._to_line_df().drop(['geometry','prev_pt'], axis=1) 
line_gdf = line_gdf.rename(columns={'line':'geometry'}).set_geometry('geometry')
line_gdf.head()

In [ ]:
line_gdf.plot(column='speed', legend=True, cmap='RdYlGn')

In [ ]:
line_gdf.hvplot(geo=True,  tiles='StamenToner', line_width=5, 
                c='speed', cmap='RdYlGn') 

In [ ]:
folium_map = folium.Map([40.02, 116.45], zoom_start=11)
folium.TileLayer('stamentoner').add_to(folium_map)
speed_scale = branca.colormap.linear.RdYlGn_09.scale(0,30)

def flip_lat_lon(geometry):
    return [(y, x) for x, y in geometry.coords]

for line, speed in line_gdf[['geometry','speed']].values:
    folium.PolyLine(
        flip_lat_lon(line),
        weight=5,
        color=speed_scale(speed)
    ).add_to(folium_map)
 
folium_map